In [45]:
!pip install unsloth

In [2]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map = "auto"
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.0,
    target_modules = ["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

Unsloth 2025.8.1 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


In [4]:
from datasets import load_dataset

dataset = load_dataset("ruslanmv/ai-medical-chatbot", split="train")
print(dataset[0].keys())

README.md:   0%|          | 0.00/863 [00:00<?, ?B/s]

dialogues.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

dict_keys(['Description', 'Patient', 'Doctor'])


In [5]:
instruction = "You are an expert medical doctor. Describe accurately to the user questions."

def format(sample):
    prompt = f"""### Instruction: {instruction}
                 ### Patient: {sample["Patient"]}
                 ### Response: {sample["Doctor"]}"""
    return {"text": prompt}

formatted_dataset = dataset.map(format)

Map:   0%|          | 0/256916 [00:00<?, ? examples/s]

In [39]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir = "./tiny-medbot",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 2,
    learning_rate = 2e-4,
    warmup_steps = 10,
    max_steps = 150,
    report_to = "none",
    logging_steps = 10,
    bf16 = False,
    fp16 = False
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_dataset,
    dataset_text_field = "text",
    max_seq_length = 512,
    packing = False,
    args = training_args,
)

trainer.train()

model.save_pretrained("./tiny-medbot")
tokenizer.save_pretrained("./tiny-medbot")

Unsloth: Tokenizing ["text"]:   0%|          | 0/256916 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 256,916 | Num Epochs = 1 | Total steps = 150
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 2 x 1) = 2
 "-____-"     Trainable parameters = 12,615,680 of 1,112,664,064 (1.13% trained)


Step,Training Loss
10,2.247900
20,2.216500
30,2.210800
40,2.293700
50,2.284600
60,2.397300
70,2.159900
80,2.269300
90,2.225700
100,2.204100


('./tiny-medbot/tokenizer_config.json',
 './tiny-medbot/special_tokens_map.json',
 './tiny-medbot/chat_template.jinja',
 './tiny-medbot/tokenizer.model',
 './tiny-medbot/added_tokens.json',
 './tiny-medbot/tokenizer.json')

In [40]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./tiny-medbot",
    max_seq_length = 1024,
    dtype = torch.float16,
    load_in_4bit = True,
)

model.eval()


==((====))==  Unsloth 2025.8.1: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Line

In [44]:
query = "Hello doctor, How long should I keep gauze in after a tooth is pulled?"
prompt = f"""### Instruction: You are an expert medical doctor. Describe accurately to the user questions.
### Patient: {query}
### Response:"""

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=150,
    temperature=0.7,
    top_k=40,
    top_p=0.9,
    repetition_penalty=1.3,
    eos_token_id=tokenizer.eos_token_id
)

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
response_only = generated_text.split("### Response:")[-1].strip()

print("Doctor:", response_only)

Doctor: Hi! Thanks for your query. Gauzes can be used as dressing or packs around teeth that have been removed from their sockets. They help prevent bleeding and promote healing of wounds by providing support and reducing swelling. The duration depends on how deep the extraction was done (how much tissue remains). If there were no significant damage to the nerve root or pulp cavity then you may need to use them only once every 24 hours until the area has fully healed. However if there was some damage to these structures then they will require more frequent application. In case of any further queries consult dentist online --> https://www.dental-care.com/dentistry/tooth
